In [1]:
import requests
from requests.exceptions import ConnectionError
from time import sleep
import json
import pandas as pd
#from tapi_yandex_direct import YandexDirect
import datetime as dt
import http.client
from io import StringIO
#from tapi_yandex_direct import YandexDirect
import numpy as np
#from pandas import json_normalize
from pandas.io.json import json_normalize
import smtplib
from email import encoders
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from bs4 import BeautifulSoup as bs
import sys
from io import StringIO


In [2]:
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/campaigns'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ''

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
clientLogin = ''

# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
           "Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto",
           # Формат денежных значений в отчете
           # "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
           # "skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
           # "skipReportSummary": "true"
           }

# Создание тела запроса
body = {

  "method": "get",
  "params": {
    "SelectionCriteria": {
      "Types": ["TEXT_CAMPAIGN", "MOBILE_APP_CAMPAIGN", "DYNAMIC_TEXT_CAMPAIGN", "CPM_BANNER_CAMPAIGN", "SMART_CAMPAIGN"],
      "States": ["ARCHIVED", "CONVERTED", "ENDED", "OFF", "ON", "SUSPENDED"],
      "Statuses": ["ACCEPTED", "DRAFT", "MODERATION", "REJECTED"],
      #"StatusesPayment": ["DISALLOWED", "ALLOWED"]
    },
    "FieldNames": ["Name", "DailyBudget"],
    }
  }

body = json.dumps(body, indent=4)

req = requests.post(ReportsURL, body, headers=headers)
#req.encoding = 'utf-8'  
text_req = req.text

In [3]:
json_data = json.loads(req.content.decode('utf-8'))
data = json_data
data


{'result': {'Campaigns': [{'DailyBudget': {'Amount': 600000000,
     'Mode': 'STANDARD'},
    'Name': 'ДПО. Гид-экскурсовод - Поиск - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. Гид-экскурсовод (около) - Поиск - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. Управление персоналом - Поиск - Около - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. Управление персоналом - Поиск - Целевая - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. Управление персоналом - РСЯ - Целевая - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. Экономика предприятий и организаций - Поиск - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. Экономика предприятий и организаций - РСЯ - Ал'},
   {'DailyBudget': {'Amount': 300000000, 'Mode': 'STANDARD'},
    'Name': 'ДПО. АХО - Целевая - По

In [4]:
normalized_json = json_normalize(data['result']['Campaigns'])
print(normalized_json)

df_with_data = pd.DataFrame(normalized_json)
df_with_data['DailyBudget'] = df_with_data['DailyBudget.Amount'] / 1000000
df_with_data = df_with_data.fillna(0)
df_with_data

NameError: name 'json_normalize' is not defined

In [5]:
# добавляю рандомный дневной бюджет

#start_df['DailyBudget'] = np.random.choice([1000, 3000, 5000, 5000, 1488, 0], start_df.shape[0])
#df_for_check = start_df
#df_for_check

In [6]:
# вывожу df если есть нулевой дневной бюджет

null_df = df_with_data.loc[df_with_data['DailyBudget'] <= 0]
null_df

,Name,DailyBudget.Mode,DailyBudget.Amount,DailyBudget


In [7]:
# если НЕ создается, то ничего не происходит и тут будет ошибка, если он создается, то из него получается html-файл
# который идет на почту

if null_df.empty:
    exit()
else:
    df = pd.DataFrame(null_df)
df = df.to_html()

NameError: name 'df' is not defined

In [ ]:
# подключение к почте

email = ""
password = ""
FROM = ""
TO   = ""
subject = "test"

In [ ]:
msg = MIMEMultipart("alternative")

# адрес отправителя
msg["From"] = FROM

# адрес получателя
msg["To"] = TO

# тема
msg["Subject"] = subject

In [ ]:
# прикладываем созданный html-файл

text = bs(df, "html.parser").text
html_part = MIMEText(df, "html")
msg.attach(html_part)
print(msg.as_string())

In [ ]:
# объявляем функцию 

def send_mail(email, password, FROM, TO, msg):
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(email, password)
    server.sendmail(FROM, TO, msg.as_string())
    server.quit()

In [ ]:
# выполняем функцию

send_mail(email, password, FROM, TO, msg)